In [4]:
import plotly.graph_objects as go
import pandas as pd


dataset = pd.read_csv("./Flights dataset.csv")
dataset = dataset[dataset['CARRIER_DELAY']> 1]
dataset.sort_values(by = 'ARR_DELAY', ascending = False, inplace = True)
dataset = dataset.head(2000)

#ordered dates of December from 1st to 31st 
days = ["1", "2", "3", "4", "5", "6", "7", "8", "9", 
        "10","11", "12", "13", "14", "15", "16", "17", "18", "19", 
        "20","21", "22", "23", "24", "25", "26", "27", "28", "29",
        "30","31"]

# make list of carriers
carriers = []
for carrier in dataset["CARRIER"]:
    if carrier not in carriers:
        carriers.append(carrier)
        
# set up the layout of the figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# layout {} section set ups 
fig_dict["layout"]["xaxis"] = {"range": [1.1, 2.9], "title": "Time of flight (min)", "type": "log"}
fig_dict["layout"]["yaxis"] = {"range": [2.2, 3.3], "title": "Arrive delay time(min)", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 2000, "redraw": False},#data moving speed
                                "fromcurrent": True, "transition": {"duration": 1200,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Date of December:",
        "visible": True,
        "xanchor": "right"
    },
    #slider moving speed
    "transition": {"duration": 1200, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# data [] section set ups
day = 1 # start from Dec 1st, each date will be dealed with following steps
for carrier in carriers:
    dataset_by_day = dataset[dataset["DAY_OF_MONTH"] == day]
    dataset_by_day_and_carr = dataset_by_day[dataset_by_day["CARRIER"] == carrier]

    data_dict = {
        "x": list(dataset_by_day_and_carr["AIR_TIME"]),
        "y": list(dataset_by_day_and_carr["ARR_DELAY"]),
        "mode": "markers",
        "text": list(dataset_by_day_and_carr["ORIGIN"]),
        "marker": {
            "sizemode": "area",
            "sizeref": 5,
            "size": list(dataset_by_day_and_carr["DISTANCE"])},
        "name": carrier
    }
    fig_dict["data"].append(data_dict)

# frame [] section set ups 
for day in days: # add date into the new dataframe from Dec.1st to 31st
    frame = {"data": [], "name": str(day)}
    for carrier in carriers:
        dataset_by_day = dataset[dataset["DAY_OF_MONTH"] == int(day)]
        dataset_by_day_and_carr = dataset_by_day[dataset_by_day["CARRIER"] == carrier]

        data_dict = {
            "x": list(dataset_by_day_and_carr["AIR_TIME"]),
            "y": list(dataset_by_day_and_carr["ARR_DELAY"]),
            "mode": "markers",
            "text": list(dataset_by_day_and_carr["ORIGIN"]),
            "marker": {
                "sizemode": "area",
                "sizeref": 5,
                "size": list(dataset_by_day_and_carr["DISTANCE"])},
            "name": carrier
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [day],#transition duration speed set up here 
        {"frame": {"duration": 1200, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 1200}}
    ],
        "label": day,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

#combine the layout{} section with animation sliders 
fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

#update new layout with title
fig.update_layout(
    title={
        'text': "Top 2000 Arrive Delayed Flights Causing by Carrier in Decmber 2009",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()